In [ ]:
%%capture
!pip install openmined_psi

In [ ]:
import syft as sy
duet = sy.join_duet(loopback=True)

In [ ]:
import openmined_psi as psi

In [ ]:
duet.store.pandas

In [ ]:
# from: https://github.com/OpenMined/PSI/blob/master/private_set_intersection/python/tests.py

In [ ]:
reveal_intersection_ptr = duet.store["reveal_intersection"]

In [ ]:
reveal_intersection = reveal_intersection_ptr.get(
    request_block=True,
    name="reveal_intersection",
    reason="Are we revealing or not?",
    timeout_secs=10,
    delete_obj=False
)
reveal_intersection

In [ ]:
c = psi.client.CreateWithNewKey(reveal_intersection)
c

In [ ]:
client_items = ["Element " + str(i) for i in range(1000)]

In [ ]:
# this lets us access the static methods on the static class for now
server_ptr = duet.openmined_psi.server()

In [ ]:
s_ptr = server_ptr.CreateWithNewKey(reveal_intersection_ptr)
s_ptr

In [ ]:
server_items_ptr = duet.store["server_items"]
server_items_ptr

In [ ]:
fpr = 1.0 / (1000000000)
setup_ptr = s_ptr.CreateSetupMessage(fpr, len(client_items), server_items_ptr)

In [ ]:
request = c.CreateRequest(client_items)
# TODO: make openmined_psi.proto_request serializable
request_ptr = request.send(duet)

In [ ]:
resp_ptr = s.ProcessRequest(request_ptr)

In [ ]:
# TODO make openmined_psi.proto_response serializable
resp = resp_ptr.get(
    request_block=True,
    name="reponse",
    reason="To get the result",
    timeout_secs=10,
)

In [ ]:
if reveal_intersection:
    intersection = c.GetIntersection(setup, resp)
    iset = set(intersection)
    for idx in range(len(client_items)):
        if idx % 2 == 0:
            assert idx in iset
        else:
            assert idx not in iset

In [ ]:
if not reveal_intersection:
    intersection = c.GetIntersectionSize(setup, resp)
    assert intersection >= (len(client_items) / 2.0)
    assert intersection <= (1.1 * len(client_items) / 2.0)

In [ ]:
duet.store.pandas